<img src="../images/logos/openradar_logo.svg" width=300 alt="Python-Jlab-logo"></img>

# Py-Art, Xradar y Wradlib

---

## Introducción
En este cuadernillo (Notebook) aprenderemos los conceptos básicos para trabajar con datos provenientes de radares meteorológicos:

1. Breve introducción a Py-Art
1. Breve introducción a Xradar
2. Breve introducción a Wradlib

## Prerequisitos
| Conceptos | Importancia | Notas |
| --- | --- | --- |
| [Introducción a Numpy](https://foundations.projectpythia.org/core/numpy.html) | Necesario | Información complementaria |
| [Introducción a Xarray](https://foundations.projectpythia.org/core/xarray.html) | Necesario | Información complementaria |
| [Matplotlib](https://foundations.projectpythia.org/core/matplotlib.html) | Necesario |  Generación de gráficas|
| [Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Necesario |  Generación de mapas|
| [Radar Cookbook](https://projectpythia.org/radar-cookbook/README.html) | Necesario | Información complementaria |

- **Tiempo de aprendizaje**: 30 minutos

### Librerías

In [ ]:
import cartopy.crs as ccrs
import cmweather
import matplotlib.pyplot as plt
import numpy as np
import pyart
import wradlib as wrl
import xradar as xd

---

## 1. `Py-Art`

* [`Py-ART`](https://arm-doe.github.io/pyart/index.html) es una librería en Python que nos permite **graficar**, **corregir** y **analizar** datos de **radares meteorológicos** de diferentes **fabricantes**, **tipo**, y **modos de operación**. El software ha sido lanzado en `GitHub` como software de **código abierto bajo** una licencia **BSD**. Se ejecuta en `Linux`, `OS` y `Windows`. 

### 1.1 Lectura de datos usando `Py-Art`

Para leer nuestros datos de radar -en el caso particular de este taller usaremos datos provenientes de radares meteorológicos con formato SIGMET- podemos usar el módulo [`pyart.io.read_sigmet`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_sigmet.html). Sin embargo, `Py-Art` tiene un módulo [`I/O`](https://arm-doe.github.io/pyart/API/generated/pyart.io.html) mucho más amplio donde se pueden leer datos de otras fuentes o tipos de radares. 

In [ ]:
radar_pa = pyart.io.read_sigmet(f"../data/CAR220809191504.RAWDSX2")

Ahora podemos consultar la información contenida dentro del objeto radar usando `rada.info('compact')`

In [ ]:
radar_pa.info("compact")

Podemos listar las **variables polarimétricas** contenidas en este archivo usando `radar.fields`

In [ ]:
list(radar_pa.fields)

La variable polarimétrica más popular en es el **factor de reflectividad (Z)** que es una medida indirecta de la intensidad y el tamaño de las gotas de lluvia. Podemos ver mas información en el objeto radar usando la notación de `diccionarios` en `Python`

In [ ]:
radar_pa.fields["reflectivity"]

### 1.2 Gráficas de reflectividad ($Z$)

Ahora podemos generar salidas gráficas usando el módulo [`pyart.graph`](https://arm-doe.github.io/pyart-docs-travis/API/generated/pyart.graph.html?highlight=graph#module-pyart.graph). Esta librería nos permite realizar gráficos con o sin georeferenciacion usando [`pyart.graph.RadarDisplay`](https://arm-doe.github.io/pyart-docs-travis/API/generated/pyart.graph.RadarDisplay.html#pyart.graph.RadarDisplay) o [`pyart.graph.RadarMapDisplay`](https://arm-doe.github.io/pyart-docs-travis/API/generated/pyart.graph.RadarMapDisplay.html#pyart.graph.RadarMapDisplay)

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(121)
display_ = pyart.graph.RadarDisplay(radar_pa)
display_.plot(
    "reflectivity",
    0,
    ax=ax,
    colorbar_label="Reflectivity (dBZ)",
    cmap="ChaseSpectral",
    vmin=-10,
    vmax=60,
    title="Referencia al centro del radar",
)

projection = ccrs.PlateCarree()
ax1 = plt.subplot(122, projection=projection)
display_ = pyart.graph.RadarMapDisplay(radar_pa)
# Extraer la latitud y longitud del radar y usarla para centrar el mapa
lat_center = round(radar_pa.latitude["data"][0], 0)
lon_center = round(radar_pa.longitude["data"][0], 0)


# Determinar los anchos
lat_ticks = np.arange(lat_center - 3, lat_center + 3, 1.5)
lon_ticks = np.arange(lon_center - 3, lon_center + 3, 1.5)

# Fijar la proyección - en este caso, usamos la proyección general PlateCarree

display_.plot_ppi_map(
    "reflectivity",
    0,
    resolution="10m",
    ax=ax1,
    lat_lines=lat_ticks,
    lon_lines=lon_ticks,
    colorbar_label="Reflectivity (dBZ)",
    cmap="ChaseSpectral",
    vmin=-10,
    vmax=60,
    title="Georeferenciado",
)

### 1.3 Gráficas de variables polarimétricas

Tambien podemos generar gráficas de otras variables como reflectividad diferencial ($Z_{DR}$), diferencial de fase ($\phi_{DP}$)

In [ ]:
fig = plt.figure(figsize=(12, 4))
display_ = pyart.graph.RadarMapDisplay(radar_pa)

ax2 = plt.subplot(121, projection=projection)
display_.plot_ppi_map(
    "differential_reflectivity",
    0,
    resolution="10m",
    ax=ax2,
    lat_lines=lat_ticks,
    lon_lines=lon_ticks,
    title=r"$Reflectividad \  diferencial$",
)

ax3 = plt.subplot(122, projection=projection)
display_.plot_ppi_map(
    "differential_phase",
    0,
    ax=ax3,
    resolution="10m",
    lat_lines=lat_ticks,
    lon_lines=lon_ticks,
    title=r"$Diferencial \ de \ fase$",
)

plt.tight_layout()

### 1.4 Otras funcionaldiades de Py-Art

Para más funcionalidades de `Py-Art` puedes revisar la [documentación oficial](https://arm-doe.github.io/pyart/) o el [Radar Cookbook](https://projectpythia.org/radar-cookbook/README.html) donde podras encontrar ejemplos y casos de uso.

## 2. `Xradar`

[`Xradar`](https://docs.openradarscience.org/projects/xradar/en/stable/), de acuerdo con la documentación oficial, "es una herramienta que nos permite incorporar los datos de **radar meteorológico** al modelo de datos `Xarray`". Básicamente, esta herramienta nos permite acceder a nuestros datos de radar usando las ventajas de `etiquetas`, `coordenadas`, y `atributos`. `Xradar` es una herramienta de **código abierto** que se basa en la colaboración de la comunidad científica y sus aportes a la misma. Se encuentra en un **estado estable** y en constante desarrollo. 

### 2.1  Lectura de datos usando `Xradar`

Al igual que `Py-Art`, esta librería tiene un módulo [`I/O`](https://docs.openradarscience.org/projects/xradar/en/stable/importers.html) que soporta datos de radares de múltiples fuentes/formatos. Para nuestro caso particular, utilizaremos el método [`xd.io.open_iris_datatree`](https://docs.openradarscience.org/projects/xradar/en/latest/notebooks/Iris.html#) para leer nuestros datos en formato SIGMET.

In [ ]:
radar_xd = xd.io.open_iris_datatree(f"../data/CAR220809191504.RAWDSX2")
display(radar_xd)

Como podemos observar nuestro objeto radar tiene una estructura de un [`xarray.datatree`](https://xarray-datatree.readthedocs.io/en/latest/data-structures.html) lo que nos permite tener múltiples elevaciones en un solo objeto.
Para acceder a nuestro `Dataset` podemos usar la notación de `diccionarios` de `Python` seguido por el método `.ds`

In [ ]:
display(radar_xd["sweep_0"].ds)

### 2.2 Gráfico de reflectividad (Z)

Como ya conocemos, `Xarray` nos permite generar gráficos de manera rápida, sin tener que utilizar la librería `matplotlib`, usando el método `.plot`

In [ ]:
radar_xd["sweep_0"]["DBZH"].plot(cmap="ChaseSpectral", vmin=-10, vmax=60)

Nuestro `Dataset` tiene por `coordenadas` y `dimensiones` el **azimuth** y el **rango**. Debemos georeferenciar nuestro `Dataset` para visualizar los datos en cordenadas relativas al radar o geográficas. Para lograr esto usamos el método [`xr.georeference`](https://docs.openradarscience.org/projects/xradar/en/stable/georeference.html)

In [ ]:
radar = radar_xd.xradar.georeference()
display(radar["sweep_0"])

Como se puede observar, **x**, **y** y **z** han sido agregados como `coordenadas` a nuestro `Dataset`. 

In [ ]:
radar_xd["sweep_0"].x

Nuevamente generemos el gráfico de reflectividad pero ahora utilizando las nuevas `coordenadas`

In [ ]:
radar_xd["sweep_0"]["DBZH"].plot(x="x", y="y", cmap="ChaseSpectral", vmin=-10, vmax=60)

### 2.3 Selección de datos (Slicing)

Como mencionamos en el tutorial de `Xarray`, podemos utilizar las coordenadas y los atributos para seleccionar datos a lo largo de las dimensiones. Seleccionemos datos de reflecividad entre  `40° < azimuth < 120°` y `0 < rango < 150km`

In [ ]:
radar_xd["sweep_0"]["DBZH"].sel(azimuth=slice(40, 120), range=slice(0, 150 * 1e3)).plot(
    x="x", y="y", cmap="ChaseSpectral", vmin=-10, vmax=60
)

También podemos visualizar la reflectividad en función del rango. Intentémoslo para `azimuth=55`

In [ ]:
radar_xd["sweep_0"]["DBZH"].sel(azimuth=55, method="nearest").plot()

### 2.4 Acceso a otras variables plorimétricas

Podemos acceder a las `variables` dentro del `Dataset` usando la notación de diccionarios de `Python` o usando el método `Punto`. Tratemos de acceder a la **reflectividad diferencial** $Z_{DR}$

In [ ]:
radar_xd["sweep_0"]["ZDR"].plot(x="x", y="y", vmin=-1, vmax=5)

In [ ]:
radar_xd["sweep_0"].RHOHV.plot(x="x", y="y", vmin=0, vmax=1, cmap="jet")

En general, `Xradar` es una librería relativamente "nueva" que sigue en constante evolución y construcción por parte de la comunidad científica. Para más información pueden consultar la [`documentación oficial`](https://docs.openradarscience.org/projects/xradar/en/stable/index.html).

## 3. `Wradlib`

[`Wradlib`](https://docs.wradlib.org/en/latest/) es una librería que, al igual que las anteriores, nos permite acceder a datos de radares meteorológicos de diversas fuentes y formatos. De acuerdo con la documentación oficial  "`Wradlib` está diseñado para ayudar en los pasos más importantes del procesamiento de datos del radar meteorológico. Estos pueden incluir: leer formatos de datos comunes, georreferenciación, convertir la reflectividad en intensidad de lluvia, identificar y corregir fuentes de error típicas (como el desorden o la atenuación) y visualizar los datos".

### 3.1 Lectura de datos

`Wradlib` ofrece un módulo [`I/O`](https://docs.wradlib.org/en/stable/io.html) completo para la lectura de archivos de radar en diferentes formatos y plataformas. En nuestro caso, utilizaremos `wr.io.read_iris` para leer nuestro archivo SIGMET

In [ ]:
radar_wrl = wrl.io.read_iris(f"../data/CAR220809191504.RAWDSX2")

In [ ]:
radar_wrl.keys()

El objeto retornado es un  [`OrderedDict`](https://docs.python.org/3/library/collections.html#collections.OrderedDict), que simplemente es un `diccionario` con unos métodos adicionales a los diccionarios normales de Python. Podemos acceder a las variables polarimétricas del radar de la siguiente manera:

In [ ]:
for variable in radar_wrl["data"][1]["ingest_data_hdrs"].keys():
    print(variable)

Si queremos acceder a la reflectividad debemos usar la llave `DB_DBZ`

In [ ]:
radar_wrl["data"][1]["sweep_data"]["DB_DBZ"]

Puede ser un poco confuso el acceso a los datos usando `Wralib`. Por lo tanto podemos usar la librería `Xradar` para acceder a los datos y utilizar los métodos de `Wradlib`. En el siguiente ejemplo accedemos al `sweep_0`, luego asignamos `coordenadas` y `georeferencia`

In [ ]:
swp = radar_xd["sweep_0"].ds.copy()
swp

In [ ]:
swp = swp.assign_coords(sweep_mode=swp.sweep_mode)
swp = swp.wrl.georef.georeference()

In [ ]:
swp

### 3.2 Gráficos usando `wrl.vis` 

Finalmente podemos generar el gráfico de reflectividad (Z) usando el módulo de visualización `wrl.vis.plot`

In [ ]:
fig = plt.figure(figsize=(5, 5))
pm = swp.DBZH.wrl.vis.plot(vmin=-10, vmax=60, cmap="ChaseSpectral")

Podemos agregar georeferenciación a nuestro `Dataset` usando el método `wrl.georef.epsg_to_osr`. Para nuestro caso utilizaremos `epsg:4326` o también llamado [`WGS84`](https://en.wikipedia.org/wiki/World_Geodetic_System). Ahora podemos ver que nuestras coordenadas `x` y `y` estan en coordenadas geográficas.

In [ ]:
epsg = wrl.georef.epsg_to_osr(4326)
swp = swp.wrl.georef.georeference(crs=epsg)
swp


Procedemos ahora a generar un gráfico adicionándole atributos como el centro del radar y unos anillos concéntricos a diferentes distancias usando [`wrl.vis.plot_ppi_crosshair`](https://docs.wradlib.org/en/latest/generated/wradlib.vis.plot_ppi_crosshair.html)

In [ ]:
fig = plt.figure(figsize=(6, 4))
pm = swp.DBZH.wrl.vis.plot(ax=111, fig=fig, cmap="ChaseSpectral", vmin=-10, vmax=60)
txt = plt.title("Simple PPI Carimagua Radar")

ax = plt.gca()
wrl.vis.plot_ppi_crosshair(
    site=(swp.longitude.values, swp.latitude.values, swp.altitude.values),
    ranges=[50e3, 150e3, 225e3],
    angles=[0, 90, 180, 270],
    line=dict(color="white"),
    circle={"edgecolor": "white"},
    ax=ax,
    crs=epsg,
);

### 3.3 Mapa de ruido (Clutter)

Entre muchas otras herramientas y funcionalidades, `Wradlib` nos permite realizar un mapa de ruidos (clutter) usando el filtro desarrollado por [Gabella et al. (2002)](https://iris.polito.it/handle/11583/1411995)

In [ ]:
clutter = swp.DBZH.wrl.classify.filter_gabella(tr1=12, n_p=6, tr2=1.1)
pm = clutter.wrl.vis.plot(cmap=plt.cm.gray)
plt.title("Clutter Map")

`Wradlib` es de librería que nos permite aplicar muchas otras técnicas de filtrado, visualización, estimación cuantitativa de la precipitación, entre otros. Para mas información pueden consultar la [documentación oficial](https://docs.wradlib.org/en/latest/index.html).

---

## Conclusiones

En este cuadernillo aprendimos sobre algunas librerías que nos permiten leer y generar productos basados en datos de radares meteorológicos. 

## Fuentes y referencias

* Helmus, J.J. & Collis, S.M., (2016). The Python ARM Radar Toolkit (Py-ART), a Library for Working with Weather Radar Data in the Python Programming Language. Journal of Open Research Software. 4(1), p.e25. DOI: http://doi.org/10.5334/jors.119
* Radar Cookbook [https://projectpythia.org/radar-cookbook/README.html] DOI[https://doi.org/10.5281/zenodo.8075855]
* Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Version v2023.05.01) https://doi.org/10.5281/zenodo.7884572